# 8. 원-핫 인코딩(One-Hot Encoding)
- 단어 집합(vocabulary)에서는 기본적으로 book과 books와 같이 단어의 변형 형태도 다른 단어로 간주
- 원-핫 인코딩을 위해서 먼저 해야할 일은 단어 집합을 만드는 일(**텍스트의 모든 단어들은 중복X**)
- 이 단어 집합에 고유한 숫자를 부여하는 정수 인코딩을 진행
- Example
    - 텍스트에 단어가 총 5,000개가 존재한다면, 단어 집합의 크기는 5,000입니다. 5,000개의 단어가 있는 이 단어 집합의 단어들마다 1번부터 5,000번까지 인덱스를 부여한다고 해보겠습니다. 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 부여할 수 있습니다.

## 1) 원-핫 인코딩(One-Hot Encoding)이란?
> 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식 $\Longrightarrow$이렇게 표현된 벡터를 **원-핫 벡터(One-Hot vector)**라고 함

- 과정
    - (1) 각 단어에 고유한 인덱스를 부여합니다. (정수 인코딩)
    - (2) 표현하고 싶은 단어의 인덱스의 위치에 1을 부여하고, 다른 단어의 인덱스의 위치에는 0을 부여합니다.

In [1]:
# Example
from konlpy.tag import Okt  
okt=Okt()  
token=okt.morphs("나는 자연어 처리를 배운다")  
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [2]:
word2index={}
for voca in token:
     if voca not in word2index.keys():
       word2index[voca]=len(word2index)
print(word2index)

# 빈도수 순대로 단어를 정렬하여 고유한 인덱스를 부여하는 작업이 사용되기도 함

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [3]:
# 토큰을 입력하면 해당 토큰에 대한 원-핫 벡터를 만들어내는 함수
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index=word2index[word]
       one_hot_vector[index]=1
       return one_hot_vector

one_hot_encoding("자연어",word2index)

[0, 0, 1, 0, 0, 0]

## 2) 케라스(Keras)를 이용한 원-핫 인코딩(One-Hot Encoding)
- **to_categorical()** : 케라스에서 지원하는 원-핫 인코딩을 수행하는 유용한 도구 

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text="나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

t = Tokenizer()
t.fit_on_texts([text])
print(t.word_index) # 각 단어에 대한 정수 인코딩 결과 출력.

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [8]:
# texts_to_sequences()를 통해서 이를 정수 시퀀스로 변환가능

# 일부만 가지고 해보기
sub_text="점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded=t.texts_to_sequences([sub_text])[0]
print(encoded)

[2, 5, 1, 6, 3, 7]


#### One-Hot Encoding해보기

In [9]:
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


## 3) 원-핫 인코딩(One-Hot Encoding)의 한계

#### (1) 이러한 표현 방식은 단어의 개수가 늘어날 수록, 벡터를 저장하기 위해 필요한 공간이 계속 늘어남(벡터의 차원이 점점 높아짐)
- 이는 저장 공간 측면에서는 매우 비효율적인 표현 방법

#### (2) 원-핫 벡터는 단어의 유사도를 표현하지 못함
- 늑대, 호랑이, 강아지, 고양이라는 4개의 단어에 대해서 원-핫 인코딩을 해서 각각, [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]이라는 원-핫 벡터를 부여받았다고 합시다. 이 때 원-핫 벡터로는 강아지와 늑대가 유사하고, 호랑이와 고양이가 유사하다는 것을 표현할 수가 없습니다.
- 단어 간 유사성을 알 수 없다는 단점은 검색 시스템 등에서 심각한 문제
    - 가령, 여행을 가려고 웹 검색창에 '삿포로 숙소'라는 단어를 검색한다고 합시다. 제대로 된 검색 시스템이라면, '삿포로 숙소'라는 검색어에 대해서 '삿포로 게스트 하우스', '삿포로 료칸', '삿포로 호텔'과 같은 유사 단어에 대한 결과도 함께 보여줄 수 있어야 합니다. 하지만 단어간 유사성을 계산할 수 없다면, '게스트 하우스'와 '료칸'과 '호텔'이라는 연관 검색어를 보여줄 수 없음
    
#### (3) 해결방법
단어의 잠재 의미를 반영하여 다차원 공간에 벡터화 하는 기법으로 크게 두 가지가 있습니다.
- (1) 카운트 기반의 벡터화 방법인 **LSA, HAL**
- (2) 예측 기반으로 벡터화하는 **NNLM, RNNLM, Word2Vec, FastText**
- (3)그리고 카운트 기반과 예측 기반 두 가지 방법을 모두 사용하는 방법으로 **GloVe**라는 방법이 존재